In [1]:
import os #interacting with operating system

import random 
import math 
import pygame 

from os import listdir
#used to get a list of all the files and directories in a specified directory 

from os.path import isfile, join 
#isfile is used to check if the file is present in the os or not 
#to join one or more paths (?)
#We are going to be dynamically loading all the sprite sheets and images 

pygame.init()

pygame.display.set_caption("Platformer")

#BG_COLOR = (255, 255, 255) #We have currently chosen white as the background colour 
WIDTH, HEIGHT = 800, 600
FPS = 60 
PLAYER_VEL = 5

window = pygame.display.set_mode((WIDTH, HEIGHT))

# path = join("Python-Platformer","assets", dir1, dir2)

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
def flip(sprites): 
    return [pyagme.transform.flip(sprite, True, False) for sprite in sprites]

def load_sprite_sheets(dir1, dir2, width, height, direction=False):
    path = join("Python-Platformer", "assets", dir1, dir2)
    images = [f for f in listdir(path) if isfile(join(path, f))]

    all_sprites = {}

    for image in images:
        sprite_sheet = pygame.image.load(join(path, image)).convert_alpha()

        sprites = []
        for i in range(sprite_sheet.get_width() // width):
            surface = pygame.Surface((width, height), pygame.SRCALPHA, 32)
            rect = pygame.Rect(i * width, 0, width, height)
            surface.blit(sprite_sheet, (0, 0), rect)
            sprites.append(pygame.transform.scale2x(surface))

        if direction:
            all_sprites[image.replace(".png", "") + "_right"] = sprites
            all_sprites[image.replace(".png", "") + "_left"] = flip(sprites)
        else:
            all_sprites[image.replace(".png", "")] = sprites

    return all_sprites

In [ ]:
def get_block(size):
    path = join("Python-Platformer","assets", "Terrain", "Terrain.png")
    image = pygame.image.load(path).convert_alpha()
    surface = pygame.Surface((size, size), pygame.SRCALPHA, 32)
    rect = pygame.Rect(96, 0, size, size)
    surface.blit(image, (0, 0), rect)
    return pygame.transform.scale2x(surface)


In [ ]:
class Player(pygame.sprite.Sprite):
    COLOR = (255, 0, 0)
    GRAVITY = 1
    SPRITES = load_sprite_sheets("MainCharacters", "MaskDude", 32, 32, True)
    ANIMATION_DELAY = 5
    
    def __init__(self, x, y, width, height):
        super().__init__()
        self.rect = pygame.Rect(x, y, width, height) 
        # rect is basically a tupple storing 4 values 
        self.x_vel = 0
        self.y_vel = 0
        self.mask = None 
        self.direction = "left" 
        self.animation_count = 0
        self.fall_count = 0
        

    def move(self, dx, dy):
         self.rect.x += dx
         self.rect.y += dy

    def move_left(self, vel):
        self.x_vel = - vel
        if self.direction != "left":
            self.direction = "left" 
            self.animation_count = 0 

    def move_right(self, vel):
        self.x_vel = vel
        if self.direction != "right":
            self.direction = "right" 
            self.animation_count = 0 

    def loop(self, fps):
        self.y_vel += min(1, (self.fall_count / fps) * self.GRAVITY)
        self.move(self.x_vel, self.y_vel)

        self.fall_count += 1

    def update_sprite(self):
        sprite_sheet = "idle"
        if self.x_vel != 0:
            sprite_sheet = "run"

        sprite_sheet_name = sprite_sheet + "_" + self.direction
        sprites = self.SPRITES[sprite_sheet_name]
        sprite_index = (self.animation_count //
                        self.ANIMATION_DELAY) % len(sprites)
        self.sprite = sprites[sprite_index]
        self.animation_count += 1
        #self.update()

    def update(self):
        self.rect = self.sprite.get_rect(topleft=(self.rect.x, self.rect.y))
        self.mask = pygame.mask.from_surface(self.sprite)


    def draw(self, win):
        self.sprite = self.SPRITES["idle_" + self.direction][0]
        win.blit(self.sprite, (self.rect.x, self.rect.y))
        
        
        

In [ ]:
class Object(pygame.sprite.Sprite):
    def __init__(self, x, y, width, height, name=None):
        super().__init__()
        self.rect = pygame.Rect(x, y, width, height)
        self.image = pygame.Surface((width, height), pygame.SRCALPHA)
        self.width = width
        self.height = height
        self.name = name

    def draw(self, win, offset_x):
        win.blit(self.image, (self.rect.x - offset_x, self.rect.y))

In [ ]:
class Block(Object):
    def __init__(self, x, y, size):
        super().__init__(x, y, size, size)
        block = get_block(size)
        self.image.blit(block, (0, 0))
        self.mask = pygame.mask.from_surface(self.image)


In [ ]:
def get_background(name):
    #pass
    image = pygame.image.load(join("Python-Platformer","assets", "Background", name))
    x, y, width, height = image.get_rect()
    tiles = []

    for i in range(WIDTH // width + 1):
        for j in range(HEIGHT // height + 1):
            pos = (i * width, j * height) #position of the top left hand corner 
            tiles.append(pos)

    return tiles, image
    

In [ ]:
def draw(window, background, bg_image, player):
    for tile in background :
        window.blit(bg_image, tile)

    player.draw(window)
    
    pygame.display.update()

In [ ]:
def handle_move(player):
    keys = pygame.key.get_pressed()

    player.x_vel = 0
    if keys[pygame.K_LEFT]:
        player.move_left(PLAYER_VEL)
    if keys[pygame.K_RIGHT]:
        player.move_right(PLAYER_VEL)

In [ ]:
def main(window): 
    clock = pygame.time.Clock()
    background, bg_image = get_background("Blue.png")

    block_size = 96 

    player = Player(100, 100, 50, 50)
    blocks = [Block(0, HEIGHT = block_size, block_size)]
    
    run = True
    while run:
        clock.tick(FPS)
        #this maintains that our while loop runs 60 FPS
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                break 
                
        player.loop(FPS)
        handle_move(player)
        draw(window, background, bg_image, player, blocks)

    pygame.quit()
    quit()
                
   
    #pass
    #pass is a null statement which can be used as placeholder for future code 
    #suppose we have a loop we don't want to execute yet then we use pass so we can use it in the future 
    

if __name__ == "__main__" :
    main(window)
    